In [1]:
!pip install seaborn
!pip install xgboost
!pip install --upgrade xgboost
!pip install catboost
!pip install pycaret #pycaret 설치
!pip install category_encoders
# !pip install --upgrade pycaret

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-pac

In [2]:
import pandas as pd
import numpy as np
from category_encoders.ordinal import OrdinalEncoder
from pycaret.classification import *
from sklearn.model_selection import train_test_split

In [3]:
df_train = pd.read_csv("train.csv")  # 학습용 데이터
df_test = pd.read_csv("submission.csv")  # 테스트 데이터(제출파일의 데이터)

In [4]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [5]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
    "is_converted",
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

# for col in label_columns:
#     df_all[col] = label_encoding(df_all[col])
for col in label_columns:
    df_train[col] = label_encoding(df_train[col])

In [6]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

In [7]:
x_train, x_val, y_train, y_val = train_test_split(
#     df_train.drop("is_converted", axis=1),
    df_train,
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

In [8]:
x_train.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
43573,0.50,//Saudi Arabia,AS,NaN,5293,NaN,Enterprise,NaN,NaN,NaN,...,LGESJ,NaN,0,0,NaN,NaN,NaN,NaN,45,False
25857,0.75,//U.A.E,ID,0.049180,11045,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEGF,less than 3 months,1,1,0.001183,0.04984,retail,Fashion,153,False
42264,0.25,//Colombia,AS,NaN,47466,NaN,Enterprise,16.0,NaN,NaN,...,LGECB,NaN,0,0,NaN,NaN,NaN,NaN,40,False
7665,1.00,/kendujhar/India,ID,NaN,12526,NaN,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,NaN,NaN,NaN,NaN,175,False
23088,1.00,/Lucknow/India,ID,0.031196,40086,NaN,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.000572,0.04863,education,NaN,175,False


In [9]:
## 모델 정의
model = setup(
    data = x_train,
    test_data = x_val,
    target = "is_converted",
    fold = 15
)

,Description,Value
0,Session id,7854
1,Target,is_converted
2,Target type,Binary
3,Original data shape,"(47439, 29)"
4,Transformed data shape,"(47439, 44)"
5,Transformed train set shape,"(33207, 44)"
6,Transformed test set shape,"(14232, 44)"
7,Ordinal features,1
8,Numeric features,13
9,Categorical features,15


In [ ]:
top5_model = compare_models(
               round=4,
               sort="Accuracy",
               n_select = 11 )

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,07:31:50
Status,. . . . . . . . . . . . . . . . . .,Fitting 15 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9612,0.9417,0.6113,0.8810,0.7213,0.7012,0.7150,2.1760
xgboost,Extreme Gradient Boosting,0.9605,0.9590,0.6387,0.8442,0.7265,0.7057,0.7141,0.5240
rf,Random Forest Classifier,0.9585,0.9490,0.5703,0.8851,0.6930,0.6719,0.6909,1.2613
knn,K Neighbors Classifier,0.9578,0.8571,0.5893,0.8528,0.6964,0.6746,0.6882,0.5913
ada,Ada Boost Classifier,0.9553,0.9391,0.5812,0.8225,0.6805,0.6573,0.6691,0.8060
et,Extra Trees Classifier,0.9454,0.9375,0.4176,0.8389,0.5567,0.5311,0.5685,1.1080
dt,Decision Tree Classifier,0.9363,0.8041,0.6449,0.6096,0.6246,0.5899,0.5915,0.3687
ridge,Ridge Classifier,0.9188,0.0000,0.0919,0.5527,0.1569,0.1362,0.2002,0.3040
lr,Logistic Regression,0.9170,0.6405,0.0007,0.0667,0.0014,-0.0002,0.0008,0.6147
lda,Linear Discriminant Analysis,0.9110,0.7931,0.2097,0.4212,0.2793,0.2375,0.2544,0.3653


Processing:   0%|          | 0/79 [00:00<?, ?it/s]